In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def click_mas():
    # Pinchar en el botón del menu "Más"
    masMenu = driver.find_element(By.XPATH, '//*[@id="content"]/header/div[2]/ul/li[5]/a')

    try:
        masMenu.click()
    except (ElementNotInteractableException, NoSuchElementException):
        # Maneja la excepción y espera antes de intentar nuevamente
        print("Anuncio detectado, reiniciando driver...")
        driver.refresh()
        time.sleep(3) 
        masMenu.click()

In [6]:
# Crea una instancia del controlador del navegador
driver = webdriver.Chrome()

# Navega a la página web que deseas hacer scraping
driver.get("https://mister.mundodeportivo.com/new-onboarding/#market")

# Espera a que se cargue la página
driver.implicitly_wait(15)

# Encuentra el botón de "Consentir" 
button = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
# Haz clic en el botón de "Consentir" 
button.click()

# Encuentra el botón de "Siguinete" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[2]/button')
# Haz clic en el botón de "Siguiente" 
button.click()
button.click()
button.click()
button.click()

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/button[3]')
button.click()

# Localiza el elemento del input gmail
inputgmail = driver.find_element(By.XPATH, '//*[@id="email"]')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputgmail.clear()

# Ingresa texto en la caja de texto
inputgmail.send_keys("m31_grupo6@outlook.com")

# Localiza el elemento del input gmail
inputpsw = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[2]/input')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputpsw.clear()

# Ingresa texto en la caja de texto
inputpsw.send_keys("Chocoflakes2")

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[3]/button')
button.click()

# Espera a que se cargue la página
driver.implicitly_wait(10)

click_mas()

# Pinchar en el botón "Jugaodres" para acceder al listado de jugadores 
jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')

try:
    jugadoresbtn.click()
except (ElementNotInteractableException, NoSuchElementException):
    # Maneja la excepción y espera antes de intentar nuevamente
    print("Anuncio detectado, reiniciando driver...")
    driver.refresh()
    time.sleep(3)
    click_mas()
    time.sleep(3)
    try:
        jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')
        jugadoresbtn.click()
    except: 
        print("Reicicia el script :(")
        sys.exit()
        
pag=2
while True:

    # Encontrar todos los elementos li
    elementos_lis = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")

    # Longitud de la lista de elementos encontrados
    length=len(elementos_lis)

    index=0
    while index < length:
        # Encontrar todos los elementos li
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        elementos_li[index].click()

        nombre = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[2]")
        apellido = driver.find_element(By.XPATH, " /html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[3]")
        print(index," -",nombre.text,apellido.text)
        
        #Retroceder página
        driver.back()
        time.sleep(2)
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        driver.execute_script("arguments[0].scrollIntoView(true);", elementos_li[index])
        time.sleep(2)
        index += 1

    #jugador cambio de pagina
    try:
        ver_mas = driver.find_element(By.XPATH, '/html/body/div[6]/div[3]/div[3]/div[1]/button')
        ver_mas.click()
        time.sleep(4)
    except:
        break

    # Encontrar todos los elementos li
    elementos_lis = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
    # Longitud de la lista de elementos encontrados
    length=len(elementos_lis)

    elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
    elementos_li[index].click()

    time.sleep(1)
 
    #Retroceder página
    driver.back()
        
    print("Sigueinte página...  (",pag,")")
    index=1
    pag+=1
    
print("Todos los jugadores scapeados")

0  - Jude Bellingham
1  - Takefusa Kubo
2  - Antoine Griezmann
3  - Aleix García
4  -  Isco
5  - Ante Budimir
6  - Iñaki Williams
7  - Artem Dovbyk
8  -  Pepelu
9  - Mikel Oyarzabal
10  - Bryan Zaragoza
11  - Borja Mayoral
12  -  Sávio
13  - Gerard Moreno
14  - Dani Carvajal
15  - Brais Méndez
16  - Álvaro Valles
17  - Vedat Muriqi
18  - Javier Guerra
19  - Yangel Herrera
20  - Álvaro Morata
21  - Martín Zubimendi
22  - Alex Remiro
23  - Álvaro García
24  - Ayoze Pérez
25  - Kirian Rodríguez
26  - Hugo Duro
27  - Fede Valverde
28  - Jonathan Bamba
29  - Gorka Guruzeta
30  -  Gavi
31  - Jørgen Strand Larsen
32  - Jeremías Ledesma
33  - Ter Stegen
34  - Unai Simón
35  - Toni Kroos
36  - Alexander Sørloth
37  - Chris Ramos
38  - David López
39  - Iago Aspas
40  - Luis Rioja
41  - Sergio Arribas
42  - Robert Lewandowski
43  - Ander Barrenetxea
44  - Stole Dimitrievski
45  - Lamine Yamal
46  - Marc Roca
47  - Filip Jörgensen
48  -  Rodrygo
49  - Isi Palazón
Sigueinte página...  ( 2 )
0  - Ó